Positive Synonym Generator. 

In [ ]:
import pandas as pd

In [ ]:
df1=pd.read_csv("Tweets.csv")
df1=df1.dropna()
df1.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


Creating a DataFrame (df_filtered) containing only negative tweets.


In [ ]:
df_filtered = df1[df1['sentiment'] =='negative']
df_filtered=df_filtered.reset_index()
  
# Print the new dataframe
print(df_filtered.head())
print('Length of dataframe:', len(df_filtered))

   index      textID                                               text  \
0      1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
1      2  088c60f138                          my boss is bullying me...   
2      3  9642c003ef                     what interview! leave me alone   
3      4  358bd9e861   Sons of ****, why couldn`t they put them on t...   
4     12  74a76f6e0a       My Sharpie is running DANGERously low on ink   

    selected_text sentiment  
0        Sooo SAD  negative  
1     bullying me  negative  
2  leave me alone  negative  
3   Sons of ****,  negative  
4     DANGERously  negative  
Length of dataframe: 7781


Calculating individual sentiment scores using VADER

In [ ]:
!pip install vaderSentiment


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 7.3 MB/s 


In [ ]:
from vaderSentiment.vaderSentiment import *
new_words = {
    '****':-2.5,
}

sid_obj = SentimentIntensityAnalyzer()
sid_obj.lexicon.update(new_words)

def sentiment_scores(text):
    sentiments=[]
    sentitext= SentiText(text)
    words_and_emoticons = sentitext.words_and_emoticons
   
    for item in words_and_emoticons:
            valence = 0
            i = words_and_emoticons.index(item)
    
    

            sentiment_dict = sid_obj.sentiment_valence(valence, sentitext, item, i, sentiments)
            

    return sentiment_dict


In [ ]:
sentence=df_filtered.text[1]
print(sentence)
print(sentiment_scores(sentence))
sentiment_scores("****")

my boss is bullying me...
[0, 0, 0, -2.9, 0]


[-2.5]

In [ ]:
df_filtered['scores']=df_filtered['text'].map(lambda x:list(sentiment_scores(x)))
df_filtered.reset_index()
df_filtered.head()
#df_filtered.insert(loc=0, column='scores', value=sentiment_scores(df_filtered['text']))



,index,textID,text,selected_text,sentiment,scores
0,1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[0, -2.833, 0, 0, -0.6, 0, 0, 0, 0, 0]"
1,2,088c60f138,my boss is bullying me...,bullying me,negative,"[0, 0, 0, -2.9, 0]"
2,3,9642c003ef,what interview! leave me alone,leave me alone,negative,"[0, 0, -0.2, 0, -1.0]"
3,4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,12,74a76f6e0a,My Sharpie is running DANGERously low on ink,DANGERously,negative,"[0, 0, 0, 0, -2.0, -1.1, 0, 0]"


In [ ]:
#converting the list of scores to a dictionary={index:score}
def dict(list):
  dict={}
  for i in range(len(list)):
    dict[i]=list[i]
  return dict
df_filtered['dictionary']=df_filtered['scores'].apply(lambda x: dict(x))
df_filtered.head()

,index,textID,text,selected_text,sentiment,scores,dictionary
0,1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[0, -2.833, 0, 0, -0.6, 0, 0, 0, 0, 0]","{0: 0, 1: -2.833, 2: 0, 3: 0, 4: -0.6, 5: 0, 6..."
1,2,088c60f138,my boss is bullying me...,bullying me,negative,"[0, 0, 0, -2.9, 0]","{0: 0, 1: 0, 2: 0, 3: -2.9, 4: 0}"
2,3,9642c003ef,what interview! leave me alone,leave me alone,negative,"[0, 0, -0.2, 0, -1.0]","{0: 0, 1: 0, 2: -0.2, 3: 0, 4: -1.0}"
3,4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: ..."
4,12,74a76f6e0a,My Sharpie is running DANGERously low on ink,DANGERously,negative,"[0, 0, 0, 0, -2.0, -1.1, 0, 0]","{0: 0, 1: 0, 2: 0, 3: 0, 4: -2.0, 5: -1.1, 6: ..."


In [ ]:
#extracting the most negative word in the sentence by sorting the dictionary and returning its index 
import operator
from collections import OrderedDict
def sort_dict(dict):
  dict2=sorted(dict.items(), key=operator.itemgetter(1))

  sorted_dict = OrderedDict()
  for k, v in dict2:
    sorted_dict[k] = v
  first_pair = next(iter((sorted_dict.items())) )
  for i in range(len(df_filtered)):
    ele=first_pair[i]
    return int(ele)


df_filtered['neg_word']=df_filtered['dictionary'].apply(lambda x:sort_dict(x))
df_filtered=df_filtered.reset_index()
df_filtered.head()


# def minimum(senti):
#   senti.keys.sort()
  
#   return senti[0]

,level_0,index,textID,text,selected_text,sentiment,scores,dictionary,neg_word
0,0,1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[0, -2.833, 0, 0, -0.6, 0, 0, 0, 0, 0]","{0: 0, 1: -2.833, 2: 0, 3: 0, 4: -0.6, 5: 0, 6...",1
1,1,2,088c60f138,my boss is bullying me...,bullying me,negative,"[0, 0, 0, -2.9, 0]","{0: 0, 1: 0, 2: 0, 3: -2.9, 4: 0}",3
2,2,3,9642c003ef,what interview! leave me alone,leave me alone,negative,"[0, 0, -0.2, 0, -1.0]","{0: 0, 1: 0, 2: -0.2, 3: 0, 4: -1.0}",4
3,3,4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: ...",0
4,4,12,74a76f6e0a,My Sharpie is running DANGERously low on ink,DANGERously,negative,"[0, 0, 0, 0, -2.0, -1.1, 0, 0]","{0: 0, 1: 0, 2: 0, 3: 0, 4: -2.0, 5: -1.1, 6: ...",4


In [ ]:
#neg=df_filtered['scores'].sort_values()
#print(neg[1][0])

# type(df_filtered['neg_word'])
# df_filtered.neg_word[0][0]

In [ ]:
# def find_word(series):
#   for i in range(len(df_filtered)):
#     ele=series[i]
#     return int(ele)
# df_filtered['ind']=df_filtered['neg_word'].apply(lambda x: find_word(x))
# df_filtered.head()
index=df_filtered['neg_word'][2]  
print(index)
print(df_filtered.text[2].split()[index])


4
alone


Most negative word in sentence 

In [ ]:
 
#   for i in range(len(df_filtered)):
#     lst=text.split()
#     index=df_filtered['neg_word'][i]
#     word=lst[index-1]
#     return word

# for i in range(len(df_filtered)):
#   df_filtered['word'][i]=df_filtered['text'][i].split()[df_filtered['neg_word'][i]]

def ext_word():
  list=[]
  for i in range(len(df_filtered)):
    lst=df_filtered['text'][i].split()
    index=df_filtered['neg_word'][i]
    list.append(lst[index])
  return list
  
#store the words in a list and assign it to a new column in the dataframe.
word_list=ext_word()
df_filtered['word']=word_list
df_filtered.tail()

,level_0,index,textID,text,selected_text,sentiment,scores,dictionary,neg_word,word
7776,7776,27464,e7eecffdc8,rec game....trying not to cry...the pain is to...,breakingg,negative,"[0, 0, 0, 0, 0, 1.702, 0, 0, 0, 0, 0, -1.7, 0,...","{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 1.702, 6: 0,...",11,lose
7777,7777,27470,778184dff1,lol i know and haha..did you fall asleep?? o...,t bored,negative,"[1.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1.1, 0, 0...","{0: 1.8, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7...",11,bored
7778,7778,27472,8f5adc47ec,http://twitpic.com/663vr - Wanted to visit the...,were too late,negative,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: ...",0,http://twitpic.com/663vr
7779,7779,27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,"[1.7, 0, 0, 0, 0, 0, 0, 0, 0, -1.3, 0, 0, 0, 0...","{0: 1.7, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7...",9,lost
7780,7780,27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.6, 0, 0, 0, 0...","{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: ...",0,I`ve


In [ ]:
df_filtered.to_csv('file1.csv')

To print Synonyms

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from nltk.corpus import wordnet
def synonyms(word):
  for i in range(len(df_filtered)):
    synonyms = []
    for syn in wordnet.synsets(word):
	    for l in syn.lemmas():
		    synonyms.append(l.name())
    return synonyms
df_filtered['syn']=df_filtered['word'].apply(lambda x:synonyms(x))
df_filtered.head()


,level_0,index,textID,text,selected_text,sentiment,scores,dictionary,neg_word,word,syn
0,0,1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[0, -2.833, 0, 0, -0.6, 0, 0, 0, 0, 0]","{0: 0, 1: -2.833, 2: 0, 3: 0, 4: -0.6, 5: 0, 6...",1,SAD,"[sad, sad, deplorable, distressing, lamentable..."
1,1,2,088c60f138,my boss is bullying me...,bullying me,negative,"[0, 0, 0, -2.9, 0]","{0: 0, 1: 0, 2: 0, 3: -2.9, 4: 0}",3,bullying,"[bullying, intimidation, strong-arm, bully, br..."
2,2,3,9642c003ef,what interview! leave me alone,leave me alone,negative,"[0, 0, -0.2, 0, -1.0]","{0: 0, 1: 0, 2: -0.2, 3: 0, 4: -1.0}",4,alone,"[alone, alone, lone, lonely, solitary, alone, ..."
3,3,4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: ...",0,Sons,"[son, boy, Son, Word, Logos]"
4,4,12,74a76f6e0a,My Sharpie is running DANGERously low on ink,DANGERously,negative,"[0, 0, 0, 0, -2.0, -1.1, 0, 0]","{0: 0, 1: 0, 2: 0, 3: 0, 4: -2.0, 5: -1.1, 6: ...",4,DANGERously,"[perilously, hazardously, dangerously]"


Drop rows 

In [ ]:
x=df_filtered[df_filtered.text.str.contains(r'[*]')]

In [ ]:
df2=(df_filtered.drop(df_filtered[df_filtered.text.str.contains(r'[*]')
                 |df_filtered.word.str.contains(r'[0-9]')
                 |df_filtered.word.str.contains(r'[@#&$%+-/*:]')
                 | df_filtered.selected_text.str.contains(r'[*]')
                 | df_filtered.syn.str.contains(r'[[]]')].index))
df2.to_csv('filez.csv')


/usr/local/lib/python3.7/dist-packages/pandas/core/strings/accessor.py:1153: FutureWarning: Possible nested set at position 1
  if regex and re.compile(pat).groups:


Score of synonym

In [ ]:
synon = df2.syn[1]
print(synon)
for i in range(len(df2.syn[1])):
  print(df2.syn[1][i])
  print(sentiment_scores(df2.syn[1][i]))

['bullying', 'intimidation', 'strong-arm', 'bully', 'browbeat', 'bullyrag', 'ballyrag', 'boss_around', 'hector', 'push_around', 'browbeat', 'bully', 'swagger', 'blustery', 'bullying']
bullying
[-2.9]
intimidation
[-1.8]
strong-arm
[0]
bully
[-2.2]
browbeat
[0]
bullyrag
[0]
ballyrag
[0]
boss_around
[0]
hector
[0]
push_around
[0]
browbeat
[0]
bully
[-2.2]
swagger
[0]
blustery
[0]
bullying
[-2.9]


In [ ]:
def sent_score(syn):
    for i in range(len(df2.syn)):
      sent_score = []
      for j in range(len(syn)):
        sent_score.append(sentiment_scores(syn[j]))
      return sent_score
df2['sent_score']=df2['syn'].apply(lambda x:sent_score(x))
df2.head()


,level_0,index,textID,text,selected_text,sentiment,scores,dictionary,neg_word,word,syn,sent_score
0,0,1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[0, -2.833, 0, 0, -0.6, 0, 0, 0, 0, 0]","{0: 0, 1: -2.833, 2: 0, 3: 0, 4: -0.6, 5: 0, 6...",1,SAD,"[sad, sad, deplorable, distressing, lamentable...","[[-2.1], [-2.1], [0], [-1.7], [-1.5], [-2.2], ..."
1,1,2,088c60f138,my boss is bullying me...,bullying me,negative,"[0, 0, 0, -2.9, 0]","{0: 0, 1: 0, 2: 0, 3: -2.9, 4: 0}",3,bullying,"[bullying, intimidation, strong-arm, bully, br...","[[-2.9], [-1.8], [0], [-2.2], [0], [0], [0], [..."
2,2,3,9642c003ef,what interview! leave me alone,leave me alone,negative,"[0, 0, -0.2, 0, -1.0]","{0: 0, 1: 0, 2: -0.2, 3: 0, 4: -1.0}",4,alone,"[alone, alone, lone, lonely, solitary, alone, ...","[[-1.0], [-1.0], [-1.1], [-1.5], [0], [-1.0], ..."
4,4,12,74a76f6e0a,My Sharpie is running DANGERously low on ink,DANGERously,negative,"[0, 0, 0, 0, -2.0, -1.1, 0, 0]","{0: 0, 1: 0, 2: 0, 3: 0, 4: -2.0, 5: -1.1, 6: ...",4,DANGERously,"[perilously, hazardously, dangerously]","[[0], [0], [-2.0]]"
5,5,13,04dd1d2e34,i want to go to music tonight but i lost my vo...,lost,negative,"[0, 0.3, 0, 0, 0, 0, 0, 0, 0, -1.3, 0, 0]","{0: 0, 1: 0.3, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7...",9,lost,"[doomed, lost, lose, lose, lose, misplace, mis...","[[-3.2], [-1.3], [-1.7], [-1.7], [-1.7], [0], ..."


In [ ]:
def sent_score(syno):
    for i in range(len(df2)):
      sent_score = []
      for j in range(len(syno)):
        sent_score.append(sentiment_scores(syno[j]))
      return sent_score
df2['sent_score']=df2['syn'].apply(lambda x:sent_score(x))
df2.head()


,level_0,index,textID,text,selected_text,sentiment,scores,dictionary,neg_word,word,syn,sent_score
0,0,1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[0, -2.833, 0, 0, -0.6, 0, 0, 0, 0, 0]","{0: 0, 1: -2.833, 2: 0, 3: 0, 4: -0.6, 5: 0, 6...",1,SAD,"[sad, sad, deplorable, distressing, lamentable...","[[-2.1], [-2.1], [0], [-1.7], [-1.5], [-2.2], ..."
1,1,2,088c60f138,my boss is bullying me...,bullying me,negative,"[0, 0, 0, -2.9, 0]","{0: 0, 1: 0, 2: 0, 3: -2.9, 4: 0}",3,bullying,"[bullying, intimidation, strong-arm, bully, br...","[[-2.9], [-1.8], [0], [-2.2], [0], [0], [0], [..."
2,2,3,9642c003ef,what interview! leave me alone,leave me alone,negative,"[0, 0, -0.2, 0, -1.0]","{0: 0, 1: 0, 2: -0.2, 3: 0, 4: -1.0}",4,alone,"[alone, alone, lone, lonely, solitary, alone, ...","[[-1.0], [-1.0], [-1.1], [-1.5], [0], [-1.0], ..."
4,4,12,74a76f6e0a,My Sharpie is running DANGERously low on ink,DANGERously,negative,"[0, 0, 0, 0, -2.0, -1.1, 0, 0]","{0: 0, 1: 0, 2: 0, 3: 0, 4: -2.0, 5: -1.1, 6: ...",4,DANGERously,"[perilously, hazardously, dangerously]","[[0], [0], [-2.0]]"
5,5,13,04dd1d2e34,i want to go to music tonight but i lost my vo...,lost,negative,"[0, 0.3, 0, 0, 0, 0, 0, 0, 0, -1.3, 0, 0]","{0: 0, 1: 0.3, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7...",9,lost,"[doomed, lost, lose, lose, lose, misplace, mis...","[[-3.2], [-1.3], [-1.7], [-1.7], [-1.7], [0], ..."


Stemming of synonyms

In [ ]:
ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

df2['Tweet_stemmed'] = df2['syn'].apply(lambda x: stemming(x))
df2.head()



,level_0,index,textID,text,selected_text,sentiment,scores,dictionary,neg_word,word,syn,sent_score,Tweet_stemmed
0,0,1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[0, -2.833, 0, 0, -0.6, 0, 0, 0, 0, 0]","{0: 0, 1: -2.833, 2: 0, 3: 0, 4: -0.6, 5: 0, 6...",1,SAD,"[sad, sad, deplorable, distressing, lamentable...","[[-2.1], [-2.1], [0], [-1.7], [-1.5], [-2.2], ...","[sad, sad, deplor, distress, lament, piti, sad..."
1,1,2,088c60f138,my boss is bullying me...,bullying me,negative,"[0, 0, 0, -2.9, 0]","{0: 0, 1: 0, 2: 0, 3: -2.9, 4: 0}",3,bullying,"[bullying, intimidation, strong-arm, bully, br...","[[-2.9], [-1.8], [0], [-2.2], [0], [0], [0], [...","[bulli, intimid, strong-arm, bulli, browbeat, ..."
2,2,3,9642c003ef,what interview! leave me alone,leave me alone,negative,"[0, 0, -0.2, 0, -1.0]","{0: 0, 1: 0, 2: -0.2, 3: 0, 4: -1.0}",4,alone,"[alone, alone, lone, lonely, solitary, alone, ...","[[-1.0], [-1.0], [-1.1], [-1.5], [0], [-1.0], ...","[alon, alon, lone, lone, solitari, alon, onli,..."
4,4,12,74a76f6e0a,My Sharpie is running DANGERously low on ink,DANGERously,negative,"[0, 0, 0, 0, -2.0, -1.1, 0, 0]","{0: 0, 1: 0, 2: 0, 3: 0, 4: -2.0, 5: -1.1, 6: ...",4,DANGERously,"[perilously, hazardously, dangerously]","[[0], [0], [-2.0]]","[peril, hazard, danger]"
5,5,13,04dd1d2e34,i want to go to music tonight but i lost my vo...,lost,negative,"[0, 0.3, 0, 0, 0, 0, 0, 0, 0, -1.3, 0, 0]","{0: 0, 1: 0.3, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7...",9,lost,"[doomed, lost, lose, lose, lose, misplace, mis...","[[-3.2], [-1.3], [-1.7], [-1.7], [-1.7], [0], ...","[doom, lost, lose, lose, lose, misplac, mislay..."


Similarity Score